# Using geometric to create GCN models

- The content of the note mainly comes from Stanford CS224W 

## Prepare for the running environment

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

In [ ]:
# specific CUDA version (cpu, cu92, cu101, cu102, cu110) and PyTorch version (1.4.0, 1.5.0, 1.6.0, 1.7.0)
# !pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
# !pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
# !pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
# !pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-geometric

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import time 
import matplotlib.pyplot as plt 
import numpy as np
import networkx as nx
from datetime import datetime
from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE

batch_size = 64
hidden_dim = 64
lr = 0.02
epoch = 200
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Create a model with GCN layers

- GCN cares about node embedding learning and graph embedding learning

In [ ]:
class GNNStack(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, task='node_embed', dropout_rate=0.25):
        super(GNNStack, self).__init__()
        self.task = task
        if not (self.task == 'node_embed' or self.task == 'graph_embed'):
            raise RuntimeError('Task type not supported, expect node_embed or graph_embed but got: ', self.task)
        self.dropout_rate = dropout_rate
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(self.build_conv_model(input_dim, hidden_dim))
        for _ in range(2):
            self.conv_layers.append(self.build_conv_model(hidden_dim, hidden_dim))
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )
        self.layer_num = 3
        
    def build_conv_model(self, input_dim, hidden_dim):
        if self.task == 'node_embed':
            return pyg_nn.GCNConv(input_dim, hidden_dim)
        else: # graph embedding
            return pyt_nn.GINConv(nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLu(),
                nn.Linear(hidden_dim, hidden_dim)
            ))
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        if data.num_node_features == 0:
            x = torch.ones(data.num_nodes, 1) # node feature init (constant)
        for layer_index in range(self.layer_num):
            x = self.conv_layers[layer_index](x, edge_index)
            embed = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
        if self.task == 'graph_embed':
            x = pyg_nn.global_mean_pool(x, batch) # pooling all nodes together to represent whole graph
        x = self.post_mp(x)
        return embed, F.log_softmax(x)
    
    def loss(self, prediction, label):
        return F.nll_loss(prediction, label) # negative log-likelihood (-LL), notice we use softmax for the prediciton

## How to make customized GCN conv layers

- The most important questions are: <br>
    - how to define the message passing method (to aggregate information from neighbours)<br>
    - how to define the nodes which should be the neighbours

In [ ]:
class CustomizedConv(pyg_nn.MessagePassing):
    def __init__(self, input_dim, output_dim):
        super(CustomizedConv, self).__init__(aggr='add') # the aggregation method
        self.linear = nn.Linear(input_dim, output_dim)
        # self.self_linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x, edge_index):
        # x shape [Num(batch size), input_dim]
        # edge_index shape [2, EdegNum] (from witch node to witch node)
        # add self-loop to adjacency matrix, nodes can point to themselves (A + I)
        edge_index, _ = pyg_utils.add_self_loop(edge_index, num_nodes=x.size(0))
        # transform into feature matrix (run through the customized covn layers)
        x = self.linear(x)
        # current graph: n by n matrix, propagate will call the 'message' function
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)
        # we can also do the self information mapping independently, this depends on how u think
        # thus we do not need the self loops, the self information will go in another way
        # edge_index, _ = pyg_utils.remove_self_loop(edge_index)
        # x_self = self.self_linear(x)
        # x = self.linear(x)
        # combine the self info into the final return
        # return x_self + self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    # x_i self embedding, x_j neighbour embedding
    def message(self, x_j, edge_index, size):
        # compute the message flow in the graph
        # x_j shape [Neighbour, ouput_dim]
        # edge start nodes and end nodes (the end node is the current targe node) 
        row, col = edge_index
        degree = pyg_util.degree(row, size[0], dtype=x_j.dtype)
        # get the D^(-1/2) to normalise the ouput information
        degree_inv_sqrt = degree.pow(-0.5)
        norm  = degree_inv_sqrt[row] * degree_inv_sqrt[col]
        return norm.view(-1, 1) * x_j
    
    def update(self, aggr_out):
        # aggr_out shape [Num, ouput_dim]
        # the aggregate layers can be defined here (after message passing)
        return aggr_out

## Define the training process

In [ ]:
def train(dataset, task, writer):
    if task == 'graph_embed':
        data_size = len(dataset)
        trainloader = DataLoader(dataset[:int(data_size * 0.8)], batch_size=batch_size, shuffle=True)
        testloader = DataLoader(dataset[int(data_size * 0.8):], batch_size=batch_size, shuffle=True)
    else:
        # node embedding need the information of the whole graph to do the message flow
        trainloader = testloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # if the nodes have no features, we init it with a constant thus it should be 1
    model = GNNStack(input_dim=max(dataset.num_node_features, 1), hidden_dim=hidden_dim, output_dim=dataset.num_classes, task=task)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for step in range(epoch):
        tot_loss = 0
        for batch in trainloader:
            embed, prediction = model(batch)
            label = batch.y
            if task == 'node_embed':
                # this just like the masks in bert
                prediction = prediction[batch.train_mask]
                label = label[batch.train_mask]
            loss = model.loss(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            tot_loss += loss.item() * batch.num_graphs
        tot_loss /= len(trainloader.dataset)
        writer.add_scalar("loss", tot_loss, step)
        if step%10 == 0:
            test_acc = test(testloader, model)
            print("Current setp {}, loss {:.4f} with test accuracy {:4f}".format(step, tot_loss, test_acc))
            writer.add_scalar("test_acc", test_acc, step)
    return model

def test(loader, model, is_validation=False):
    model.eval()
    correct_num = 0
    for data in loader:
        # stop compute the gradient during propagation
        with torch.no_grad():
            embed, prediction = model(data)
            prediction = prediction.argmax(dim=1)
            label = data.y
        if model.task == "node_embed":
            mask = data.val_mask if is_validation else data.test_mask
            prediction = prediction[mask]
            label = label[mask]
        correct_num += prediction.eq(label).sum().item()
    tot = len(loader.dataset) if model.task is 'graph_embed' else 0
    if tot is 0:
        for data in loader.dataset:
            tot += torch.sum(data.test_mask).item()
    return correct_num / tot

In [ ]:
get_ipython().system_raw(
    'temsorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format('./log')
)
get_ipython().system_raw(
    './ngrok http 6006 &'
)
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys,stdin)['tunnels'][0]['public_url'])"

## Load some sample datasets and try out the GCN model

In [ ]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d"))
dataset = TUDataset(root='/tmp/IMDB-BINARY', name='IMDB-BINARY')
dataset = dataset.shuffle()
task = 'graph_embed'
model = train(dataset, task, writer)

In [ ]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d"))
dataset = Planetoid(root='/tmp/citeseer', name='citeseer')
task = 'node_embed'
model = train(dataset, task, writer)

In [ ]:
color = ['red', 'orange', 'green', 'blue', 'purple', 'black']
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
embeddings = []
colors = []
model.eval()
for batch in loader:
    embed, prediction = model(batch)
    embeddings.append(embed)
    colors += [color[y] for y in batch.y]
embeddings = torch.cat(embeddings, dim=0)
xs, ys = zip(*TSNE().fit_transform(embeddings.detach().numpy()))
plt.scatter(xs, ys, color=colors)